In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pdb
from keras.utils import to_categorical
from scipy.fftpack import fft
from scipy.io import wavfile
from sklearn.preprocessing import LabelEncoder
from python_speech_features import mfcc

## Data used: Speech Commands Data Set v0.01

In [ ]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
DATA_DIR = os.path.join(ROOT_DIR, 'data_asr', 'data')
DATA_INFO = os.path.join(ROOT_DIR, 'data_asr', 'data_info')

In [ ]:
data_list = []

In [ ]:
test_files = pd.read_csv(os.path.join(DATA_INFO, 'testing_list.txt'), sep = ' ', header = None)[0].tolist()
val_files = pd.read_csv(os.path.join(DATA_INFO, 'validation_list.txt'), sep = ' ', header = None)[0].tolist()

In [ ]:
test_lab = [os.path.dirname(i) for i in test_files]
val_lab = [os.path.dirname(i) for i in val_files]

In [ ]:
test_list = [os.path.join(DATA_DIR, f) for f in test_files if f.endswith('.wav')]
val_list = [os.path.join(DATA_DIR, f) for f in val_files if f.endswith('.wav')]

In [ ]:
for root, dirs, files in os.walk(DATA_DIR):
    data_list += [root + '/'+ f for f in files if f.endswith('.wav')]

In [ ]:
train_list = list(set(data_list) - set(test_list) - set(val_list))

In [ ]:
train_lab = [os.path.basename(os.path.dirname(i)) for i in train_list]

In [ ]:
lab = LabelEncoder()

train_encode = lab.fit_transform(train_lab)
y_encode = lab.fit_transform(train_encode)

val_encode = lab.fit_transform(val_lab)
yval_encode = lab.fit_transform(val_encode)

test_encode = lab.fit_transform(test_lab)
ytest_encode = lab.fit_transform(test_encode)

In [ ]:
ytrain = to_categorical(y_encode)
yval = to_categorical(yval_encode)
ytest = to_categorical(ytest_encode)

In [ ]:
(ytrain.shape, yval.shape, ytest.shape)

In [ ]:
np.save('ytrain.npy', ytrain)

In [ ]:
np.save('yval.npy', yval)

In [ ]:
np.save('ytest.npy', ytest)

In [ ]:
def wav2np(wav_list):
    
    samp_list, x_data = [], []
    
    for i in wav_list:
        samp_, x = wavfile.read(i)
        samp_list.append(samp_)
        x_data.append(x)
        
    return samp_list, x_data

In [ ]:
samp_train, x_train = wav2np(train_list)
samp_val, x_val = wav2np(val_list)
samp_test, x_test = wav2np(test_list)

In [ ]:
#Returns mfcc features of shape (98, 40)
def mfcc_gen(arr):
    return mfcc(i, winlen = 0.03, numcep = 40, nfilt = 40)

In [ ]:
#Zero padding function for data with less than 16000 samples
def len_pad(arr):

    if len(arr) == 98:
        return arr
    else:
        stack = (98 - len(arr), 40)     
        return np.vstack((arr, np.zeros(stack)))

In [ ]:
def data_preprocess(x):
    
    feature = np.asarray([])
    for i in x:
        mfccs = mfcc_gen(i)
        np.append(feature, len_pad(mfccs))
        
    return feature

## Load and pre-process array individually to save memory

In [ ]:
train_feat = data_preprocess(x_train)

In [ ]:
np.save('train_feat.npy', train_feat)

In [ ]:
val_feat = data_preprocess(x_val)

In [ ]:
np.save('val_feat.npy', val_feat)

In [ ]:
test_feat = data_preprocess(x_test)

In [ ]:
np.save('test_feat.npy', test_feat)